# Краткое описание
Разработка программы, которая обучает искусственную нейронную сеть (персептрон) распознавать два или более черно-белых изображения.

## 1. Цель работы
Изучить принципы работы и алгоритм обучения простейших искусственных нейронных сетей (НС).

## 2. Порядок выполнения работы
1. Изучить теоретическое введение.
2. Сформировать обучающую выборку из 10+ изображений.
3. Разработать компьютерную программу (среда разработки выбирается студентом самостоятельно).
4. Провести серию из 5+ испытаний с различными исходными данными, выявить ограничения и недостатки однослойных НС для решения задач распознавания.
5. Оформить отчет по лабораторной работе.

## 3. Требования к исходным данным и функциональности компьютерной программы
- В программе должна быть реализована возможность задания обучающей выборки из внешних файлов изображений.
- Изображения должны быть черно-белыми (bitmap) и размером не менее 9 (3x3) пикселей.
- Программа должна иметь два режима работы: обучения и распознавания.
- Обучение должно производиться по стандартному алгоритму обучения персептрона с использованием дельта-правила.
- В программе должны задаваться следующие настройки:
  - количество входов нейрона, которое соответствует общему числу пикселей изображения
  - коэффициент скорости обучения (если его значение постоянно)
  - правильные варианты элементов обучающей выборки
  - размер ошибки, при котором обучение персептрона завершается (опционально)

### На экранной форме режима обучения должны отображаться:
- элементы обучающей выборки (изображения)
- настройки алгоритма обучения
- текущие (итоговые) веса нейронов и значение порога активационной функции
- протоколы результатов обучения (значения весов для каждой итерации)

### На экранной форме режима распознавания должны отображаться:
- распознаваемое изображение (должно выбираться из всего множества)
- результат распознавания
- веса нейронов и значение порога активационной функции
- значения выходов всех нейронов до и после применения активационной функции

## 4. Рекомендации по реализации
- Для задания различной размерности распознаваемых изображений можно пользоваться одним типо-размером с максимальной разрешающей способностью, но при этом считывать только часть пикселей (например, от верхнего левого угла).
- Для решения задач обучения двухмерное изображение N*M можно преобразовывать в одномерный вектор (массив) размерностью K=N*M.
- При распознавании цветных изображений (RGB) каждому пикселю соответствует 3-х байтовая последовательность (24 входа).

## 5. Содержание отчета
- Название и цель работы
- Задание, краткое описание предметной области и выбранной задачи
- Блок-схема алгоритмов обучения и распознавания
- Протоколы проведенных экспериментов (5+), представленные в форме таблиц и графиков (допускаются скриншоты в случае программной реализации этой функциональности)
- Выводы и рекомендации по использованию НС для решения задач распознавания

# Выполнение задания

Выполнил: Журавлев Д. А. Группа 211-321 

Учебный курс: Методы работы с большими данными

In [241]:
from PIL import Image
import numpy as np
import os
import csv

np.random.seed(6340)

TRAINING_IMAGES_DIR = "./materials/nn1/images/"
WEIGHTS_DIR = "./materials/nn1/weights/"
IMAGE_SIZE = (50, 50)
LEARNING_RATE = 0.1
MIN_ERROR = 0.035


In [232]:
def load_and_vectorize_images(directory, image_size):
    train_vectors = []
    control_vectors = []
    for filename in os.listdir(directory):
        image = Image.open(os.path.join(directory, filename)).convert("L").resize(image_size)
        vector = (np.array(image) < 128).astype(int).flatten()
        group, name = str(filename).rsplit(".", 1)[0].split("_")
        data_entry = (group, f"{group}_{name}", vector)
        if name == "control":
            control_vectors.append(data_entry)
        else: 
            train_vectors.append(data_entry)
    return train_vectors, control_vectors

train_data, control_data = load_and_vectorize_images(TRAINING_IMAGES_DIR, IMAGE_SIZE)

group_labels = { y : x for x, y in enumerate( set( group[0] for group in train_data ) ) }

print(f"Train data {train_data}\nControl data {control_data}\nGroup labels {group_labels}")

Train data [('cross', 'cross_23', array([0, 0, 0, ..., 0, 0, 0])), ('polygon', 'polygon_15', array([0, 0, 0, ..., 0, 0, 0])), ('polygon', 'polygon_14', array([0, 0, 0, ..., 0, 0, 0])), ('cross', 'cross_22', array([0, 0, 0, ..., 0, 0, 0])), ('rectangle', 'rectangle_1', array([0, 0, 0, ..., 0, 0, 0])), ('rectangle', 'rectangle_3', array([0, 0, 0, ..., 0, 0, 0])), ('cross', 'cross_21', array([0, 0, 0, ..., 0, 0, 0])), ('rectangle', 'rectangle_2', array([0, 0, 0, ..., 0, 0, 0])), ('cross', 'cross_25', array([0, 0, 0, ..., 0, 0, 0])), ('polygon', 'polygon_13', array([0, 0, 0, ..., 0, 0, 0])), ('polygon', 'polygon_12', array([0, 0, 0, ..., 0, 0, 0])), ('cross', 'cross_24', array([0, 0, 0, ..., 0, 0, 0])), ('rectangle', 'rectangle_5', array([0, 0, 0, ..., 0, 0, 0])), ('oval', 'oval_10', array([0, 0, 0, ..., 0, 0, 0])), ('polygon', 'polygon_11', array([0, 0, 0, ..., 0, 0, 0])), ('rectangle', 'rectangle_4', array([0, 0, 0, ..., 0, 0, 0])), ('oval', 'oval_8', array([0, 0, 0, ..., 0, 0, 0])), ('s

In [235]:
def create_labeled_data(group_labels: dict, data: list):
    vectors = []
    labels = []
    
    for group, _, vector in data:
        vectors.append(np.array(vector))
        labels.append(group_labels[group])
    
    return np.array(vectors), np.eye(len(group_labels), dtype=int)[labels]

train_vectors, train_labels = create_labeled_data(group_labels, train_data)
control_vectors, control_labels = create_labeled_data(group_labels, control_data)

print(f"Train vectors\n {train_vectors}\nTrain labels\n {train_labels}\n Control vectors\n {control_vectors}\n Contrpl labels\n {control_labels}")

Train vectors
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Train labels
 [[1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 1 0 0]]
 Control vectors
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
 Contrpl labels
 [[0 0 0 1 0]
 [0 0 0 0 1]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]]


In [242]:
class Perceptron:
    def __init__(self, input_size, output_size, learning_rate):
        self.input_size = input_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.weights = np.random.randn(input_size, output_size)
        self.bias = np.random.randn(output_size)

    def activation(self, x):
        return 1 / (1 + np.exp(-x))

    def activation_derivative(self, x):
        return x * (1 - x)

    def train(self, X, y, epochs=10000):
        for epoch in range(epochs):
            output = self.activation(np.dot(X, self.weights) + self.bias)
            error = y - output
            d_output = error * self.activation_derivative(output)
            self.weights += np.dot(X.T, d_output) * self.learning_rate
            self.bias += np.sum(d_output, axis=0) * self.learning_rate
            loss = np.mean(np.abs(error))

            if epoch % 10000 == 0: 
                print(f"Epoch {epoch}, Loss: {loss}")

            if np.mean(np.abs(error)) <= MIN_ERROR:
                print(f"Epoch {epoch}, Loss: {loss}: Mininal error achived")
                break
        
        with open(WEIGHTS_DIR + 'weights.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(self.weights)

        with open(WEIGHTS_DIR + 'bias.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(self.bias)

    def predict(self, X):
        output = self.activation(np.dot(X, self.weights) + self.bias)
        return np.argmax(output)
    
perceptron = Perceptron(
    IMAGE_SIZE[0] * IMAGE_SIZE[1],
    len(group_labels.keys()),
    LEARNING_RATE
)

In [243]:

perceptron.train(train_vectors, train_labels, epochs=100000)

Epoch 0, Loss: 0.43717475759866026
Epoch 10000, Loss: 0.04962329212412908
Epoch 20000, Loss: 0.041153374922060226
Epoch 30000, Loss: 0.040946993998940105
Epoch 40000, Loss: 0.040826391426408266
Epoch 50000, Loss: 0.04074355731210883
Epoch 60000, Loss: 0.04068250308888625
Epoch 62628, Loss: 0.034604508618664084: Mininal error achived


Error: iterable expected, not numpy.float64

In [226]:

def calc_accuracy(vectors, labels):
    correct_predictions = 0
    for vector, label in zip(vectors, labels):
        prediction = perceptron.predict(vector)
        predicted_group = [ i for i in group_labels if group_labels[i]==prediction ][0]
        expected_group = [ i for i in group_labels if group_labels[i]==np.where(label == 1)[0][0] ][0]
        prediction_correction = "✅" if expected_group == predicted_group else "❌"
        print(f"{prediction_correction} Prediction: {predicted_group}, expected: {expected_group}")
        if expected_group == predicted_group:
            correct_predictions += 1

    return correct_predictions / len(vectors) * 100

print(f"Точность на обучающих данных: {calc_accuracy(train_vectors, train_labels):.2f}%\n")
print(f"Точность на контрольных данных: {calc_accuracy(control_vectors, control_labels):.2f}%\n")

✅ Prediction: cross, expected: cross
✅ Prediction: polygon, expected: polygon
✅ Prediction: polygon, expected: polygon
✅ Prediction: cross, expected: cross
✅ Prediction: rectangle, expected: rectangle
❌ Prediction: oval, expected: rectangle
✅ Prediction: cross, expected: cross
✅ Prediction: rectangle, expected: rectangle
✅ Prediction: cross, expected: cross
✅ Prediction: polygon, expected: polygon
✅ Prediction: polygon, expected: polygon
✅ Prediction: cross, expected: cross
✅ Prediction: rectangle, expected: rectangle
✅ Prediction: oval, expected: oval
✅ Prediction: polygon, expected: polygon
✅ Prediction: rectangle, expected: rectangle
✅ Prediction: oval, expected: oval
❌ Prediction: polygon, expected: star
✅ Prediction: star, expected: star
✅ Prediction: oval, expected: oval
❌ Prediction: oval, expected: star
✅ Prediction: oval, expected: oval
✅ Prediction: star, expected: star
✅ Prediction: star, expected: star
✅ Prediction: oval, expected: oval
Точность на обучающих данных: 88.00%
